# Assignment 1 - License plate locator

## Description

Automatically locate the number plate in the following image. (Available in numberplates2020.zip from Blackboard). You may try a 2D cross-correlation with a template matching the plate border, but you will probably need to ‘chamfer’ the template by convolving with a Gaussian or some other blurring function. 
You’ll also need a good edge detector such as the Canny Edge detector. Test your method on the other example car images from numberplates2020.zip and show the results. 

In your report discuss methods used, problems encountered, performance, and possible solutions. Comment on the problems encountered in plate extraction and the difficulties in designing a general plate extractor.

##### Marking Scheme
* Coding of a solution to locate one number plate, description and explanation of method, related images and graphs (5 marks)
* Testing method on all other number plates, modification of method as required, showing related images with detection overlay (2 marks)
* Comment on the many challenges to number plate detection including lighting, different shaped plates, different coloured plates, perspective distortion, angle of rotation, and so on and suggest possible solutions. (3 marks) 

## Introduction to algorithm

Firstly I'll introduce the different methods I've worked with and the algorithm flow step by step.
Then I'll run the algorithm to show the results for car 1. I will discuss why some methods were chosen and some abandoned for this case. Next I'll show how the algorithm works for the rest of the images, and discuss why the results are variyng.



## Step 1 - Imports

I will import the necessary libraries and need a function to import the image I want to process. 
I'm using the [OpenCV](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_table_of_contents_imgproc/py_table_of_contents_imgproc.html) library for Python to do image processing. Numpy is needed for mathematical operations and pyplot to plot results.

The image is imported in colors so I can use the orignal for drawing the countours in the later steps.


In [52]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def import_image(image):
    """Imports the given image in colors"""
    return cv2.imread(image, cv2.IMREAD_COLOR)


## Step 2 - Preprocessing

Further we introduce different ways of preprocessing the images. 

#### Grayscale
First, you would like to convert your image to grayscale - this is done to reduce information in each pixel, and we keep it to only two dimensions.

#### Thresholding
Thresholding helps classifying pixels in the images, given the threshold value. There are several types of thresholds available, and I did try out both simple thresholding, adaptive thresholding and Otsu thresholding. 

#### Blur
Bluring is used to remove noise from the image and smooth things out, using a low-pass filter kernel. The kernel size can be chosen. There are many options for different blur methods, but Gaussian is used in this case.

#### Resizing and cropping
Images may come in varying sizes and therefore it would be nice to change the size of the images I'm processing.
The crop methods divides the image into 10 both horizontally and vertically, and crop the image removing top 2/10 and bottom 1/10 of the height, and 1/10 of both left and right side from the width.

#### Morphology
Methods used for extracting image components that are useful in the representation and description of region shape. I've implemented methods for erosion, dilation, opening and closing.

#### Remove shadows
A function to remove shadows from the images, which includes using many of the already implemented methods.
This method include:
* Dilate the image, in order to get rid of the text.
* Blur the image, this gives a good background image that contains shadows.
* Calculate the difference between the original and the background. 
* Normalize the image, so that we use the full dynamic range.
* Threshold the image using Truncated threshold, and then normalize again




In [53]:
def grayscaled(image):
    """Turns an image into a grayscaled image"""
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def threshold(image, thresh=170, maxval=255, threshold_type=cv2.THRESH_TOZERO):
    ret, image_thresh = cv2.threshold(image, thresh, maxval, threshold_type)
    return image_thresh

def adaptive_threshold(image, maxval=255, adaptive_threshold_type=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, threshold_type=cv2.THRESH_TOZERO, block_size=11, c=2):
    return cv2.adaptiveThreshold(image, maxval, adaptive_threshold_type, threshold_type, block_size, c)

def blur(image, kernel_tuple=(5,5)):
    return cv2.GaussianBlur(image, kernel_tuple, 0)

def resize(image, size=None, scale_x=2, scale_y=2, interpolation=cv2.INTER_LINEAR):
    return cv2.resize(src=image, dsize=size, fx=scale_x, fy=scale_y, interpolation=interpolation)
    
def crop(image):
    image_tuple = image.shape
    height, width = image_tuple[0], image_tuple[1]
    return image[height*2//10 : height*9//10, width//10 : width*9//10]

def morph_opening(image, kernel_size=(5,5)):
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

def morph_closing(image, kernel_size=(5,5)):
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

def remove_shadows(image):
    dilated_img = cv2.dilate(image, np.ones((5,5), np.uint8)) 
    cv2.imshow("dil", dilated_img)
    
    blurred_img = blur(dilated_img, (3,3))
    cv2.imshow("blur", blurred_img)
    
    diff_img = 255 - cv2.absdiff(image, blurred_img)
    cv2.imshow("diff", diff_img)
    
    norm_img = diff_img.copy() 
    cv2.normalize(diff_img, norm_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    cv2.imshow("norm", norm_img)
    
    thr_img = threshold(norm_img, 230, 0, cv2.THRESH_TRUNC)
    cv2.imshow("thresh", thr_img)
    cv2.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    cv2.imshow("normed_thresh", thr_img)
    cv2.waitKey()
    return thr_img

def dilation(image, kernel_size=(5,5), iterations = 1):
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.dilate(image, kernel, iterations)

def erosion(image, kernel_size=(5,5), iterations = 1):
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.erode(image, kernel, iterations)


## Step 3 - Edge detector

Using an edge detector would be useful for finding the egdes of the license plates. Here, Canny is used with the option to set min and max values; those who lie between these two thresholds are classified edges or non-edges based on their connectivity. Canny is used because it automatically removes noises first, unlike Sobel and Laplacian. 
The Canny edge detector in OpenCV is using 5x5 Gaussian filter to reduce noise and non-maximum suppression to remove unwanted pixels.

In [54]:

def edge_detector(image, minval=100, maxval=150):
    return cv2.Canny(image, minval, maxval)


## Step 4 - Finding the license plate

After the preprocessing is done, we can draw countours to identify the license plates.

The find_countours method takes the preprocessed image and the original image and find countours in the image. This is represented as a list. Then I draw all the possible countours in the image (in green).

The mehtod find_license_plate is then used. The list is here sorted on area size, and only the top n entries are included. Every countour calculates an approximation. It approximates a contour shape to another shape with less number of vertices depending upon the precision I specify, which is the epsilon. An epsilon of 0.05 indicates 5% of the arc length. 

I iterate through all the top n countours and check one by one if the contour contains four corners, as that would most likely be the number plate. 

If there is four corners, it is checked agains several other methods.
Some countours marked the whole image as a big box, which gave a perfect square. To avoid this from being the "best" rectangle, countour_is_whole_image is used.
Some countours had four corners, but was more vertical than horizontal. Given that all license plates are horisontal rectangles, we could also exclude these countours using is_vertical_countour which calculates of the vertical lines are longer than the horizontal ones.
Some circular approximations where also marked with four corners (strange enough), and to avoid this I calculated the difference in height and width to be sure I did not mark a circle as the license plate.

I also included a method for convex hull. The convex hull function checks a curve for convexity defects and corrects it. 


In [55]:
def countour_is_whole_image(countour, image):
    left_upper_corner_width = countour[0][0][0]
    left_upper_corner_height = countour[0][0][1]
    right_lower_corner_width = countour[2][0][0]
    right_lower_corner_height = countour[2][0][1]
    if left_upper_corner_width < 20 \
        and left_upper_corner_height < 20 \
        and image.shape[0] - right_lower_corner_height < 20 \
        and image.shape[1] - right_lower_corner_width < 20:
        return True
    return False

def is_vertical_countour(countour):
    left_upper_corner_width = countour[0][0][0]
    right_upper_corner_width = countour[3][0][0]
    horizontally = abs(right_upper_corner_width - left_upper_corner_width)
    
    left_upper_corner_height = countour[0][0][1]
    left_lower_corner_height = countour[3][0][1]
    vertically = abs(left_lower_corner_height - left_upper_corner_height)
    if vertically > horizontally:
        return True
    return False

def is_rectangle(countour):
    
    left_upper_corner_height = countour[0][0][0]
    right_upper_corner_height = countour[1][0][0]
    left_upper_corner_width = countour[0][0][1]
    left_lower_corner_width = countour[3][0][1]
    
    diff_height = abs(right_upper_corner_height - left_upper_corner_height)
    diff_width = abs(left_lower_corner_width - left_upper_corner_width)
    print(diff_height, diff_width)
    if diff_height < 20 and diff_width < 20:
        return True
    return False

def convex(cnt):
    return cv2.convexHull(cnt)

def find_license_plate(image, countours, image_name, top_n=20):
    countour = sorted(countours, key = cv2.contourArea, reverse = True)[:top_n]
    screen_countour = None #store the number plate contour
    copied_image = image.copy()
    
    for c in countour:
        copy = copied_image.copy()
        copy2 = copy.copy()
        copy3 = copy.copy()
        cv2.drawContours(copy, [c], -1, (0, 255, 0), 3)
        
        #con = convex(c)
        #cv2.drawContours(copy2, [con], -1, (0, 255, 0), 3)
        #cv2.waitKey()
        
        perimeter = cv2.arcLength(c, True)
        approximation = cv2.approxPolyDP(c, 0.05*perimeter, True)
        if len(approximation) == 4: # Contures with 4 corners
            screen_countour = approximation
            cv2.drawContours(copy3, [screen_countour], -1, (0, 255, 0), 3)
            #cv2.imshow("passed", copy3)
            #cv2.waitKey()
            
            #if not is_rectangle(approximation): continue
            if countour_is_whole_image(screen_countour, copied_image): continue 
            #if is_vertical_countour(screen_countour): continue
            break

    cv2.drawContours(copied_image, [screen_countour], -1, (0, 255, 0), 3)
    return copied_image
    
    
def find_countour(original_image, preprocessed_image, image_name, color_tuple=(0,255,0)):
    countour, _ = cv2.findContours(preprocessed_image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    resized = resize(original_image)
    cropped = crop(resized)
    cropped_duplicate = cropped.copy()
    
    cv2.drawContours(cropped_duplicate, countour, -1, color_tuple, thickness=3)
    cv2.imshow("All countours", cropped_duplicate)
    #cv2.imwrite('./all_countours_'+str(image_name), cropped_duplicate)
    cv2.waitKey()
    
    return find_license_plate(cropped.copy(), countour, image_name, 10)

## Step 5 - Run algorithm

I'll use the premade methods from previous steps in preprocessing and locate_license_plate to locate the license plate in image "car1.jpg".

#### Preprocessing
Doing it this way makes it easy to see and change each step to my need. I could easily add morphing or shadow removal to the preprocessing, and change the order of each step. 

##### Steps in preprocessing for car1
* Convert to grayscale 
* Resizing and cropping
    * Working with the images for a long time, I came to the conclusion that on a general basis all images of license plates will have noise close to the image boundary, and that the license plate would almost always be located in the middle of the image, or in the bottom low part. 
    * Cropping the image based on this, removes a lot of uninteresting parts from the image that we know for a fact is not the license plate. You could also argue that an image with the license plate close to the image boundaries is a bad sample.
* Blur and Threshold 
    * Because this image has quite good lightning and a clear difference in color between car and license plate, the threshold work good for car1. The TOZERO threshold gets the job done here, but BINARY is also an option.

#### Locate_license_plate
I found that thresholding worked well for some cases, while edge detector was suitable for other images. Therefore the method was divided into either using edge detector or thresholding. Using the two together gave me overall bad results.

##### Steps in license plate locating for car1
* Blur and Threshold 
    * Because this image has quite good lightning and a clear difference in color between car and license plate, the threshold work good for car1. The TOZERO threshold gets the job done here, but BINARY is also an option.
* Finding countours
    * I got really good results with using the basic countour finder. Using the convex hull also gave me basically the same results.

In [61]:
def preprocessing(image):
    gray = grayscaled(image)
    resized = resize(gray)
    cropped = crop(resized)
    #no_shadows = remove_shadows(cropped)
    return cropped


def locate_license_plate(image_name, edge=False):
    image = import_image(image_name)
    preprocessed_image = preprocessing(image)
    
    cv2.imshow("Cropped image", preprocessed_image)
    cv2.imwrite('./preprocessed_'+str(image_name), preprocessed_image)
    cv2.waitKey()
    
    if edge:
        edges = edge_detector(preprocessed_image)
        cv2.imshow("Edge detector results", edges)
        #cv2.imwrite('./edges_'+str(image_name), edges)
        cv2.waitKey()
        countours = find_countour(image, edges, image_name)
    else:
        blurred = blur(preprocessed_image)
        thresh = threshold(blurred)
        cv2.imshow("Thresholded image", thresh)
        cv2.imwrite('./thresholded_'+str(image_name), thresh)
        cv2.waitKey()
        countours = find_countour(image, thresh, image_name)
        
    cv2.imshow("Final image with plate detected", countours)
    #cv2.imwrite('./result_'+str(image_name), countours)
    cv2.waitKey()


## Results

### Original image
<img src="car1.jpg" width="500">

### Preprocessed image
Here we see that the image is cropped and we have removed parts of the picture that is not interesting for our task, like the Maserati logo etc.
<img src="preprocessed_car1.jpg" width="500">

### Thresholded image
After blurring and using TOZERO threshold, it is significantly easier to detect the license plate
<img src="thresholded_car1.jpg" width="500">

### All countours
Here is a representation of all the countours the algorithm found in the image
<img src="all_countours_car1.jpg" width="500">

### Final result
The final result is shown with the best fitting approximated countour
<img src="result_car1.jpg" width="500">

## Testing on the other cars

The same algorithm is now performed on all the other images.
It turns out this detects three cars in total.


In [13]:
cars = ['car1.jpg', 'car2.jpg', 'car3.jpg', 'car4.jpg', 'car5.jpg', 'car6.jpg', 'car7.png', 'car8.jpg']

for car in cars:
    locate_license_plate(car)

## Results

Including car 1, this algorithm detects car2 and car4 as well.


### Original image
Success
<img src="car2.jpg" width="300"> 
<img src="car3.jpg" width="300"> 
<img src="car4.jpg" width="300">
<img src="car5.jpg" width="300">
<img src="car6.jpg" width="300"> 
<img src="car7.png" width="300"> 
<img src="car8.jpg" width="300"> 



### Final result
Success
<img src="result_car2.jpg" width="300">
Fail
<img src="result_car3.jpg" width="300">
Success
<img src="result_car4.jpg" width="300">
Fail
<img src="result_car5.jpg" width="300">
Fail
<img src="result_car6.jpg" width="300">
Fail
<img src="result_car7.png" width="300">
Fail
<img src="result_car8.jpg" width="300">


## Testing out other methods 

By changing the algorithm, I can obtain better results for some cars.
For car 6 and car 7, combining the `remove_shadow` using the **Canny edge detector** is the way to go.

The `remove_shadow` step in now included after the image is cropped, like so:
```python
def preprocessing(image):
    gray = grayscaled(image)
    resized = resize(gray)
    cropped = crop(resized)
    no_shadows = remove_shadows(cropped) # The change
    return no_shadows
```

#### The other cars
Provided the methods I've implemented, I've unsuccessfully managed to detect the license plate for the other cars: car3, car5 and car8. See more in discussion.


In [62]:
locate_license_plate('car1.jpg')
#locate_license_plate('car7.png', edge=True)

### Results car 6 and car 7
Result from the edge detector
<img src="edges_car6.jpg" width="400">
<img src="edges_car7.png" width="400">
Final result
<img src="edges_result_car6.jpg" width="400">
<img src="edges_result_car7.jpg" width="400">

# Discussion


## Results
This is not a general (enough) method for detecting all license plates out there. I implemented the techniques we learned and discussed in class, but there is room for improvement. You could always keep tweeking some parameters like kernel size or epsilon to gain better results, but I felt applying the different processing methods and see what they did was more educational than finding the perfect sweetspot for some values and watch how that affected the overall result.

There were many trials and errors, and in the end the simple solution with cropping, blurring and threshold was the one giving best results for most pictures at a time.

## Challenges with lisence plate detection
Working with this assignment gives a good understanding to what kind of challenges there is in license plate detection.

**Colors on signs:**
Out of the eight images provided, there are many different signs. Firstly they come in different colors; white, yellow and black. Especially the black sign on a red/darker car is hard to work with. When the license plate is the same color as the car/background, thresholding will normally do no good. My method is therefor not very good for the red Ferrari (car5).

**Lightning and shadows:**
The reason I created a `remove_shadows` method was because it became clear that the shadows in the images was creating a lot of trouble. Like discussed in class, thresholding is a very good method if it is done in environment with **controlled illumination**, which is mostly not the case for these kind of pictures. The edge detector was not that good either on the images with a lot of shadows, like car3.

**Size and resolution:** 
There is no guarantee that the samples are of high quality. The images provided here are all in different sizes, either closeups of the plate like car1 and car2, but also images with a lot of unimportat infomation. That is why I chose to resize and crop the images, and assume that all license plates will probably be located in the cropped area. While running my algorithm, the `find_countours` method detected the rear or front window as possible license plates, because they were rectangles. By cropping out the top of the car, you reduce the chance of this. 

## Alternative methods

**Template matching:**
An alternative method is template matching. I was long considering this as an alternative, but chose not to implement it for several reasons. First of all, I would have to choose one sample to match with. If that was a white license plate, it would be hard to detect the black one. Also the ratio of length and width of the license plates are varying in the samples given, and the images are all different sizes. I think template matching would be great if it was given that all plates were in one format.

**Classification of plates:** Another method is to classify the images based on their characteristics, eg. all white plates are treated in some way, white yellow plates are treated in another way. That would prove for a more complex and probably better algorithm but also way more time consuming to create. 

**Enhance the countour algorithm:** I faced some problems with the countour algorithm. Eg. for car7, it found a countour around the license plate using my standard algorithm, but for some reason it was not considered an approximation with four corners. Understanding and tweeking the contour algorithm could prove better results as well. There were a lot of times when I think the preprocessing did a good job, but the countours were dissapointing. Another way could be to fill the top_n countours and paint that on the image, and then find the plates. Unfortunately, I did not find a good way to achieve this.
